### LIBRARY INSTALLATION

In [68]:
 !pip install pmdarima
 !pip install skforecast
 !pip install awswrangler
 !pip install holidays


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [69]:
# IMPORTING LIBRARIES

# ======================================================================================
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pickle
import time
import os
from io import StringIO
import boto3
import contextlib
import re
import matplotlib.pyplot as plt
import awswrangler as wr
import holidays


plt.style.use('seaborn-darkgrid')

from io import BytesIO
import joblib

# pmdarima
from pmdarima import ARIMA
from pmdarima import auto_arima

# statsmodels
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

# skforecast
from skforecast.Sarimax import Sarimax
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection_sarimax import backtesting_sarimax
from skforecast.model_selection_sarimax import grid_search_sarimax
from sklearn.metrics import mean_absolute_error

import warnings

### ABT LOAD

In [70]:
# Set up S3 client
client = boto3.client('s3')
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
path = 'ABTv3/' # Folder under analysis

In [71]:
##Conexion a ABT##
# Listing objects 
elements = client.list_objects(Bucket=bucket_name, Prefix=path)
# Initialize an empty list 
dfs = []
# Iterate through each CSV in the S3 path
for obj in elements.get('Contents', []):
#    print(obj)
    # Check if the object is a CSV file
    if obj['Key'].endswith('.csv'):
        # Read CSV content from S3
        response = client.get_object(Bucket=bucket_name, Key=obj['Key'])
        print(obj['Key'])
        csv_content = response['Body'].read().decode('utf-8')

        # Transform CSV content to df
        df = pd.read_csv(StringIO(csv_content))

        dfs.append(df)
final_df = pd.concat(dfs, ignore_index=True)


ABTv3/ABTv3.csv


In [72]:
#Date format
final_df['date'] = pd.to_datetime(final_df['date'])

In [73]:
# Create a boolean mask to filter by country and date
mask = (final_df['country'] == 'MEXICO') & (final_df['date'].dt.month == 11) & (final_df['date'].dt.day == 2)
# Mark the Day of the Dead according to the mask
final_df['day_of_the_dead'] = 0  # Initialize with 0
final_df.loc[mask, 'day_of_the_dead'] = 1  # Mark as 1 where the mask is True

In [74]:
#Transformo fecha
final_df['date'] = pd.to_datetime(final_df['date']).dt.date

In [75]:
# Acoto fechas
final_df = final_df[final_df['date'] < pd.to_datetime('2023-12-19').date()]

### PAYER DEFINITION

In [76]:
payer = 'BANRURAL (RYT)_GUATEMALA' # 
payer_key = 'BPI'

In [77]:
# Celda de inicio
start_time = time.time()

In [78]:
#### SETEO EL PAYER ####
datos=final_df.loc[final_df.payer_country==payer]
datos = datos.set_index('date')

In [79]:
# datos = final_df[final_df['payer'].str.contains(payer_key)]
# datos = datos.set_index('date')

In [80]:
#final_df[final_df['payer'].str.contains(payer_key)]['date'].min()

### DEFINITION OF VARIABLES TO BE USED

In [81]:
datos = datos.asfreq('D')

In [82]:
#datos.columns.to_list()

In [83]:
datos = datos[['amount', 
                'var_rate_lag_2','var_rate_lag_3','var_rate_lag_4', 'var_rate_lag_5',
                'var_rate_lag_6','var_rate_lag_7',
                'var_rate_lag_8','var_rate_lag_9',
                'var_rate_lag_10','var_rate_lag_11','var_rate_lag_12','var_rate_lag_13', 
                'var_rate_lag_14','var_rate_lag_15','var_rate_lag_16','var_rate_lag_17',
               'var_rate_lag_18','var_rate_lag_19','var_rate_lag_20','var_rate_lag_21',
               'var_rate_lag_22','var_rate_lag_23','var_rate_lag_24','var_rate_lag_25',
                'var_rate_lag_26','var_rate_lag_27','var_rate_lag_28','var_rate_lag_29', 
               'ratio_coupon_tx_lag_2', 
                'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 
                'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_7', 
                'ratio_coupon_tx_lag_8', 
                'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 
                'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14',
                'ratio_coupon_tx_lag_15','ratio_coupon_tx_lag_16','ratio_coupon_tx_lag_17', 
                'ratio_coupon_tx_lag_18','ratio_coupon_tx_lag_19','ratio_coupon_tx_lag_20',
                'ratio_coupon_tx_lag_21','ratio_coupon_tx_lag_22','ratio_coupon_tx_lag_23',
                'ratio_coupon_tx_lag_24','ratio_coupon_tx_lag_25','ratio_coupon_tx_lag_26',
                'ratio_coupon_tx_lag_27','ratio_coupon_tx_lag_28','ratio_coupon_tx_lag_29',
               'tx_lag_2', 'tx_lag_3','tx_lag_4', 'tx_lag_5','tx_lag_6', 'tx_lag_7', 
                'tx_lag_8', 'tx_lag_9', 'tx_lag_10', 'tx_lag_11','tx_lag_12', 'tx_lag_13',
               'tx_lag_14', 'tx_lag_15', 'tx_lag_16','tx_lag_17', 'tx_lag_18','tx_lag_19',
                'tx_lag_20', 'tx_lag_21',
                 'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3', 
                'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 
               'var_tx_cancelled_lag_7', 
               'var_tx_cancelled_lag_8', 'var_tx_cancelled_lag_9', 
                'var_tx_cancelled_lag_10', 'var_tx_cancelled_lag_11', 'var_tx_cancelled_lag_12', 
                'var_tx_cancelled_lag_13', 'var_tx_cancelled_lag_14','var_tx_cancelled_lag_15', 
                'var_tx_cancelled_lag_16', 'var_tx_cancelled_lag_17', 'var_tx_cancelled_lag_18',
                'var_tx_cancelled_lag_19', 'var_tx_cancelled_lag_20','var_tx_cancelled_lag_21', 
                'var_tx_cancelled_lag_22','var_tx_cancelled_lag_23', 'var_tx_cancelled_lag_24',
                'var_tx_cancelled_lag_25', 'var_tx_cancelled_lag_26','var_tx_cancelled_lag_27', 
                'var_tx_cancelled_lag_28', 'var_tx_cancelled_lag_29',
                'margin_lag_2', 'margin_lag_3', 'margin_lag_4', 'margin_lag_5',
                'margin_lag_6', 'margin_lag_7', 
                'margin_lag_8', 'margin_lag_9', 'margin_lag_10', 
               'is_holiday',
               'is_fourth_of_july',
               'var_30ds',
               'christmas_day',
               'new_year_day',
               'post_holiday',
               'day_of_the_dead'
              ]]

In [84]:
# Workaround para payers nuevos
# split_index = int(len(datos) * 0.5)

### TRAINING, VALIDATION AND TESTING PERIODS

In [85]:
# Train-validation-test
# ======================================================================================
fin_train = '2023-03-31'
fin_val = '2023-06-21'
print(
    f"Fechas entrenamiento : {datos.index.min()} --- {datos.loc[:fin_train].index.max()}  "
    f"(n={len(datos.loc[:fin_train])})"
)
print(
    f"Fechas validacion    : {datos.loc[fin_train:].index.min()} --- {datos.loc[:fin_val].index.max()}  "
    f"(n={len(datos.loc[fin_train:fin_val])})"
)
print(
    f"Fechas test          : {datos.loc[fin_val:].index.min()} --- {datos.index.max()}  "
    f"(n={len(datos.loc[fin_val:])})"
)


Fechas entrenamiento : 2021-01-01 00:00:00 --- 2023-03-31 00:00:00  (n=820)
Fechas validacion    : 2023-03-31 00:00:00 --- 2023-06-21 00:00:00  (n=83)
Fechas test          : 2023-06-21 00:00:00 --- 2023-12-18 00:00:00  (n=181)


### DEFINITION OF EXOGENOUS VARIABLES

In [86]:
exogenas=[
       'var_rate_lag_2','var_rate_lag_3','var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6','var_rate_lag_7',
       'var_rate_lag_8','var_rate_lag_9',
               'var_rate_lag_10','var_rate_lag_11','var_rate_lag_12','var_rate_lag_13', 
                'var_rate_lag_14','var_rate_lag_15','var_rate_lag_16','var_rate_lag_17',
                'var_rate_lag_18','var_rate_lag_19','var_rate_lag_20','var_rate_lag_21',
                'var_rate_lag_22','var_rate_lag_23','var_rate_lag_24','var_rate_lag_25',
                'var_rate_lag_26','var_rate_lag_27','var_rate_lag_28','var_rate_lag_29', 
                'ratio_coupon_tx_lag_2', 
                'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 
                 'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_7', 
                'ratio_coupon_tx_lag_8', 
               'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 
                'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14',
                'ratio_coupon_tx_lag_15','ratio_coupon_tx_lag_16','ratio_coupon_tx_lag_17', 
                'ratio_coupon_tx_lag_18','ratio_coupon_tx_lag_19','ratio_coupon_tx_lag_20',
                'ratio_coupon_tx_lag_21','ratio_coupon_tx_lag_22','ratio_coupon_tx_lag_23',
                'ratio_coupon_tx_lag_24','ratio_coupon_tx_lag_25','ratio_coupon_tx_lag_26',
                'ratio_coupon_tx_lag_27','ratio_coupon_tx_lag_28','ratio_coupon_tx_lag_29',
                'tx_lag_2', 'tx_lag_3','tx_lag_4', 'tx_lag_5','tx_lag_6', 'tx_lag_7', 
                'tx_lag_8', 'tx_lag_9', 'tx_lag_10', 'tx_lag_11','tx_lag_12', 'tx_lag_13',
                'tx_lag_14', 'tx_lag_15', 'tx_lag_16','tx_lag_17', 'tx_lag_18','tx_lag_19',
                'tx_lag_20', 'tx_lag_21',
                'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3', 
                'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 
                'var_tx_cancelled_lag_7', 
                 'var_tx_cancelled_lag_8', 'var_tx_cancelled_lag_9', 
                'var_tx_cancelled_lag_10', 'var_tx_cancelled_lag_11', 'var_tx_cancelled_lag_12', 
                'var_tx_cancelled_lag_13', 'var_tx_cancelled_lag_14','var_tx_cancelled_lag_15', 
                'var_tx_cancelled_lag_16', 'var_tx_cancelled_lag_17', 'var_tx_cancelled_lag_18',
                'var_tx_cancelled_lag_19', 'var_tx_cancelled_lag_20','var_tx_cancelled_lag_21', 
                'var_tx_cancelled_lag_22','var_tx_cancelled_lag_23', 'var_tx_cancelled_lag_24',
                'var_tx_cancelled_lag_25', 'var_tx_cancelled_lag_26','var_tx_cancelled_lag_27', 
                'var_tx_cancelled_lag_28', 'var_tx_cancelled_lag_29',
                'margin_lag_2', 'margin_lag_3', 'margin_lag_4', 'margin_lag_5',
               'margin_lag_6', 'margin_lag_7', 
               'margin_lag_8', 'margin_lag_9',  'margin_lag_10', 
                'is_holiday',
               'is_fourth_of_july',
               'var_30ds',
               'christmas_day',
               'new_year_day',
               'post_holiday',
               'day_of_the_dead'
              ]

In [87]:
datos[exogenas] = datos[exogenas].fillna(0)

### SELECTION OF SIGNIFICANT EXOGENOUS VARIABLES

In [88]:
# Modelo SARIMAX con skforecast.Sarimax
# ==============================================================================
warnings.filterwarnings("ignore", category=UserWarning, message='Non-invertible|Non-stationary')
modelo = Sarimax(order=(2, 1, 1), seasonal_order=(2, 1, 1, 7))
modelo.fit(y=datos['amount'].loc[:fin_train], exog=datos[exogenas].loc[:fin_train] )
modelo.summary()
#warnings.filterwarnings("default")

/opt/conda/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                     
=========================================================================================
Dep. Variable:                            amount   No. Observations:                  820
Model:             SARIMAX(2, 1, 1)x(2, 1, 1, 7)   Log Likelihood              -11227.217
Date:                           Wed, 13 Mar 2024   AIC                          22708.434
Time:                                   19:30:32   BIC                          23305.271
Sample:                               01-01-2021   HQIC                         22937.550
                                    - 03-31-2023                                         
Covariance Type:                             opg                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
var_rate_lag_2          -6.982e+05   1446.713   -482.606      0.000   -7.01e+05   -6.95e+05
var_rate_lag_3           1.289e+05   6547.014     19.691      0.000    1.16e+05    1.42e+05
var_rate_lag_4          -8.655e+05   2469.374   -350.487      0.000    -8.7e+05   -8.61e+05
var_rate_lag_5           4.735e+05   4651.004    101.807      0.000    4.64e+05    4.83e+05
var_rate_lag_6            8.52e+05   3485.127    244.475      0.000    8.45e+05    8.59e+05
var_rate_lag_7           1.089e+06   3718.442    292.998      0.000    1.08e+06     1.1e+06
var_rate_lag_8           9.411e+05   2676.599    351.604      0.000    9.36e+05    9.46e+05
var_rate_lag_9           2.719e+05   3399.161     79.982      0.000    2.65e+05    2.79e+05
var_rate_lag_10          1.201e+06   3535.474    339.673      0.000    1.19e+06    1.21e+06
var_rate_lag_11          1.156e+06   5056.399    228.612      0.000    1.15e+06    1.17e+06
var_rate_lag_12           2.33e+06   5237.021    444.998      0.000    2.32e+06    2.34e+06
var_rate_lag_13          2.266e+06   4653.913    486.950      0.000    2.26e+06    2.28e+06
var_rate_lag_14          2.207e+06   3210.427    687.581      0.000     2.2e+06    2.21e+06
var_rate_lag_15          2.056e+06   4371.851    470.265      0.000    2.05e+06    2.06e+06
var_rate_lag_16          1.414e+06   2369.009    596.766      0.000    1.41e+06    1.42e+06
var_rate_lag_17          2.008e+06   5822.402    344.930      0.000       2e+06    2.02e+06
var_rate_lag_18          1.412e+06   3340.004    422.886      0.000    1.41e+06    1.42e+06
var_rate_lag_19          1.515e+06   5829.298    259.844      0.000     1.5e+06    1.53e+06
var_rate_lag_20          1.916e+06   4679.704    409.431      0.000    1.91e+06    1.93e+06
var_rate_lag_21          1.836e+06   2530.439    725.526      0.000    1.83e+06    1.84e+06
var_rate_lag_22          1.762e+06   3556.354    495.332      0.000    1.75e+06    1.77e+06
var_rate_lag_23          1.484e+06   2348.040    632.120      0.000    1.48e+06    1.49e+06
var_rate_lag_24          2.008e+06   1725.604   1163.604      0.000       2e+06    2.01e+06
var_rate_lag_25          1.047e+06   5857.570    178.786      0.000    1.04e+06    1.06e+06
var_rate_lag_26          1.594e+06   4286.599    371.822      0.000    1.59e+06     1.6e+06
var_rate_lag_27           1.36e+06   3471.205    391.693      0.000    1.35e+06    1.37e+06
var_rate_lag_28          1.504e+06   2683.145    560.391      0.000     1.5e+06    1.51e+06
var_rate_lag_29          2.694e+05   3503.257     76.897      0.000    2.63e+05    2.76e+05
ratio_coupon_tx_lag_2    3.331e+05   4343.363     76.681      0.000    3.25e+05    3.42e+05
ratio_coupon_tx_lag_3    5.893e+05   4825.184    122.125      0.000     5.8e+05    5.99e+05
ratio_coupon_tx_lag_4    2.109e+05   3349.387     62.963      0.000    2.04e+05    2.17e+05
ratio_coupon_t

In [89]:
#with open(f'sarimax_summary_{payer}_7d_12_03.txt', 'w') as f:
    #f.write(modelo.summary().as_text())_

In [90]:
#### !pip install lxml
# Obtener la tabla de resumen como cadena
tabla_resumen = modelo.summary().tables[1].as_html()
# Convertir la cadena HTML en un DataFrame de pandas
df_resumen = pd.read_html(tabla_resumen, header=0, index_col=0)[0]
# Filtrar el DataFrame para quedarte solo con las filas que contienen valores p
df_valores_p = df_resumen[df_resumen.index.str.contains('P>|z|')]
# Definir el umbral de significancia
umbral_significancia = 0.05
# Filtrar las filas donde el valor p sea menor que el umbral de significancia
variables_significativas = df_valores_p[df_valores_p['P>|z|'] < umbral_significancia]
indices_variables_significativas = variables_significativas.index.tolist()
# Convertir los índices en una lista de caracteres
exogenas_total = list(map(str, indices_variables_significativas))
# Lista de exogenas filtrada
exogenas = [var for var in exogenas_total if not var.startswith(('ar.', 'ma.', 'sigma'))]
# Mostrar la lista de exogenas filtrada
print(exogenas)

['var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13', 'var_rate_lag_14', 'var_rate_lag_15', 'var_rate_lag_16', 'var_rate_lag_17', 'var_rate_lag_18', 'var_rate_lag_19', 'var_rate_lag_20', 'var_rate_lag_21', 'var_rate_lag_22', 'var_rate_lag_23', 'var_rate_lag_24', 'var_rate_lag_25', 'var_rate_lag_26', 'var_rate_lag_27', 'var_rate_lag_28', 'var_rate_lag_29', 'ratio_coupon_tx_lag_2', 'ratio_coupon_tx_lag_3', 'ratio_coupon_tx_lag_4', 'ratio_coupon_tx_lag_5', 'ratio_coupon_tx_lag_6', 'ratio_coupon_tx_lag_7', 'ratio_coupon_tx_lag_8', 'ratio_coupon_tx_lag_9', 'ratio_coupon_tx_lag_10', 'ratio_coupon_tx_lag_11', 'ratio_coupon_tx_lag_12', 'ratio_coupon_tx_lag_13', 'ratio_coupon_tx_lag_14', 'ratio_coupon_tx_lag_15', 'ratio_coupon_tx_lag_16', 'ratio_coupon_tx_lag_17', 'ratio_coupon_tx_lag_18', 'ratio_coupon_tx_lag_19', 'ratio_coupon_t

/tmp/ipykernel_20/4020151013.py:5: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_resumen = pd.read_html(tabla_resumen, header=0, index_col=0)[0]


In [91]:
# Abro el archivo en modo escritura, con la opción 'append' para agregar contenido al final del archivo
#with open(f'sarimax_summary_{payer}_7d_8lags.txt', 'a') as f:
with open(f'sarimax_summary_{payer}_2d.txt', 'a') as f:
    # Escribir el contenido de la variable 'exogenas' en el archivo
    f.write('\n\nContenido de la variable "exogenas":\n')
    f.write(str(exogenas))

### SEARCH THE BEST MODEL 

In [92]:
# Auto arima: seleccion basada en AIC
# ==============================================================================
resultados_autoarima = auto_arima(
            y                 = datos['amount'].loc[:fin_val],
            exog              = datos[exogenas].loc[:fin_val],
            start_p           = 0,
            start_q           = 0,
            max_p             = 7,
            max_q             = 7,
            seasonal          = True,
            #test              = 'adf',
            m                 = 7, # periodicidad de la estacionalidad
            #d                 = 1, # El algoritmo determina 'd' (lo obligo porque claramente es no estacionario)
            #D                 = 1, # El algoritmo determina 'D' (idem arriba)
            max_d             = 2,
            max_D             = 2,
            trace             = True,
            error_action      = 'ignore',
            suppress_warnings = True,
            stepwise          = True
        )
opt_order_2 = resultados_autoarima.order
opt_seasonal_order_2 = resultados_autoarima.seasonal_order

Performing stepwise search to minimize aic
 ARIMA(0,1,0)(1,0,1)[7] intercept   : AIC=26646.966, Time=0.29 sec
 ARIMA(0,1,0)(0,0,0)[7] intercept   : AIC=27389.361, Time=0.02 sec
 ARIMA(1,1,0)(1,0,0)[7] intercept   : AIC=26667.066, Time=0.16 sec
 ARIMA(0,1,1)(0,0,1)[7] intercept   : AIC=26928.274, Time=0.17 sec
 ARIMA(0,1,0)(0,0,0)[7]             : AIC=27387.369, Time=0.02 sec
 ARIMA(0,1,0)(0,0,1)[7] intercept   : AIC=26931.411, Time=0.20 sec
 ARIMA(0,1,0)(1,0,0)[7] intercept   : AIC=26676.599, Time=0.08 sec
 ARIMA(0,1,0)(2,0,1)[7] intercept   : AIC=26643.557, Time=0.96 sec
 ARIMA(0,1,0)(2,0,0)[7] intercept   : AIC=26666.151, Time=0.50 sec
 ARIMA(0,1,0)(2,0,2)[7] intercept   : AIC=26644.826, Time=1.07 sec
 ARIMA(0,1,0)(1,0,2)[7] intercept   : AIC=26642.842, Time=0.64 sec
 ARIMA(0,1,0)(0,0,2)[7] intercept   : AIC=26799.854, Time=0.39 sec
 ARIMA(1,1,0)(1,0,2)[7] intercept   : AIC=26632.647, Time=0.96 sec
 ARIMA(1,1,0)(0,0,2)[7] intercept   : AIC=26793.178, Time=0.43 sec
 ARIMA(1,1,0)(1,0,1

### MODEL BACKTESTING 

In [93]:
forecaster_autoarima = ForecasterSarimax(
                 regressor=Sarimax(order=opt_order_2, seasonal_order=opt_seasonal_order_2)
             )
metrica_autoarima, predicciones_autoarima = backtesting_sarimax(
                                forecaster            = forecaster_autoarima,
                                y                     = datos['amount'],
                                exog                  = datos[exogenas],
                                initial_train_size    = len(datos.loc[:fin_val]),
                                steps                 = 2,
                                metric                = 'mean_absolute_percentage_error',
                                refit                 = False,
                                n_jobs                = 'auto',
                                suppress_warnings_fit = True,
                                verbose               = False,
                                show_progress         = True
                            )
# Predicción
nueva_fecha = datetime.strptime(fin_val, '%Y-%m-%d') + timedelta(days=1)
predicciones_autoarima = predicciones_autoarima['pred']

# Combinar los dos DataFrames en uno
resultados_predichos_autoarima= pd.DataFrame({'date': predicciones_autoarima.index, 'valor_real': datos['amount'].loc[nueva_fecha:], 'valor_predicho': predicciones_autoarima})
# Calcular el MAPE
resultados_predichos_autoarima['mape'] = abs((resultados_predichos_autoarima['valor_real'] - resultados_predichos_autoarima['valor_predicho']) / resultados_predichos_autoarima['valor_real']) * 100
resultados_predichos_autoarima['error_abs'] = abs((resultados_predichos_autoarima['valor_real'] - resultados_predichos_autoarima['valor_predicho']))
# Calcular el promedio de la columna 'mape'
promedio_mape_autoarima = resultados_predichos_autoarima['mape'].mean()
promedio_error_abs_autoarima = resultados_predichos_autoarima['error_abs'].mean()
# Imprimir el promedio
print("Promedio de la columna 'mape' - 2 dias:", promedio_mape_autoarima)
print("Promedio de la columna 'error_abs' - 2 dias:", promedio_error_abs_autoarima)

  0%|          | 0/90 [00:00<?, ?it/s]

Promedio de la columna 'mape' - 2 dias: 8.071095956009911
Promedio de la columna 'error_abs' - 2 dias: 322485.38785392675


### FIT

In [94]:
forecaster_autoarima.fit(y=datos['amount'],exog=datos[exogenas] ,suppress_warnings=True)

In [95]:
forecaster_autoarima

ForecasterSarimax 
Regressor: Sarimax(6,1,1)(0,0,2)[7] 
Regressor parameters: {'concentrate_scale': False, 'dates': None, 'disp': False, 'enforce_invertibility': True, 'enforce_stationarity': True, 'freq': None, 'hamilton_representation': False, 'maxiter': 50, 'measurement_error': False, 'method': 'lbfgs', 'missing': 'none', 'mle_regression': True, 'order': (6, 1, 1), 'seasonal_order': (0, 0, 2, 7), 'simple_differencing': False, 'sm_fit_kwargs': {}, 'sm_init_kwargs': {}, 'sm_predict_kwargs': {}, 'start_params': None, 'time_varying_regression': False, 'trend': None, 'trend_offset': 1, 'use_exact_diffuse': False, 'validate_specification': True} 
fit_kwargs: {} 
Window size: 1 
Transformer for y: None 
Transformer for exog: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9'

In [96]:
# Abro el archivo en modo escritura, con la opción 'append' para agregar contenido al final del archivo
with open(f'sarimax_summary_{payer}_7d.txt', 'a') as f:
    # Escribir el contenido de la variable 'exogenas' en el archivo
    f.write('\n\nContenido de la variable "exogenas":\n')
    f.write(str(forecaster_autoarima))

In [97]:
modelo_final = forecaster_autoarima
resultados_predichos = resultados_predichos_autoarima
mape_final = promedio_mape_autoarima

In [98]:
# Crear una carpeta llamada 'resultados' si no existe
folder_name = 'Last_releases/'
carpeta_resultados = path+folder_name+payer+'/'
print(carpeta_resultados)

try:
    client.put_object(Bucket=bucket_name, Key=(carpeta_resultados+'/'))
    print(f"Carpeta '{carpeta_resultados}' creada en el bucket '{bucket_name}'.")
except Exception as e:
    print(f"Error al crear la carpeta: {e}")
    

ABTv3/Last_releases/BANRURAL (RYT)_GUATEMALA/
Carpeta 'ABTv3/Last_releases/BANRURAL (RYT)_GUATEMALA/' creada en el bucket 'viamericas-datalake-dev-us-east-1-283731589572-analytics'.


### SAVES THE RESULTS IN AN EXCEL FILE AND THE MODEL IN A PICKLE FILE

In [99]:
#GUARDO RESULTADOS EN EXCEL

nombre_archivo_excel = os.path.join(carpeta_resultados, 'predicciones_test_2d_'+payer+'_2d'+'.xlsx')
# Guarda el DataFrame en un objeto BytesIO como archivo Excel
#excel_buffer = BytesIO()
#resultados_predichos.to_excel(excel_buffer, index=False)
# Guarda el DataFrame en un archivo XLSX en S3 usando AWS Data Wrangler
wr.s3.to_excel(resultados_predichos, f"s3://{bucket_name}/{nombre_archivo_excel}", index=False)


nombre_archivo_pickle = os.path.join(carpeta_resultados, 'MODEL_2d_'+payer+'_2d'+'.pkl')
# Guarda el modelo en un archivo pickle temporal
modelo_temp_file = 'MODEL_2d_'+payer+'.pkl'
joblib.dump(modelo_final, modelo_temp_file)

# Sube el archivo pkl a S3
try:
    wr.s3.upload(modelo_temp_file, f"s3://{bucket_name}/{nombre_archivo_pickle}")
    print(f"Modelo guardado como pkl en S3: {nombre_archivo_pickle}")
    os.remove(modelo_temp_file) # Elimina el temporal
except Exception as e:
    print(f"Error al subir el modelo a S3: {e}")


urllib3.connectionpool WARNING Connection pool is full, discarding connection: viamericas-datalake-dev-us-east-1-283731589572-analytics.s3.amazonaws.com. Connection pool size: 10
urllib3.connectionpool WARNING Connection pool is full, discarding connection: viamericas-datalake-dev-us-east-1-283731589572-analytics.s3.amazonaws.com. Connection pool size: 10


Modelo guardado como pkl en S3: ABTv3/Last_releases/BANRURAL (RYT)_GUATEMALA/MODEL_2d_BANRURAL (RYT)_GUATEMALA_2d.pkl


sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f564b003d60>
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7f56b93437c0>


In [100]:
#TXT
file_name = f'sarimax_summary_{payer}_2d.txt'
ruta_txt = os.path.join(carpeta_resultados, file_name)
# Cargar el archivo en S3
s3 = boto3.resource("s3")
s3.Bucket(bucket_name).upload_file(file_name, ruta_txt)
os.remove(file_name) # Elimina el temporal

In [101]:
# Celda de finalización
end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución total: {elapsed_time} segundos")

Tiempo de ejecución total: 407.97764682769775 segundos
